# Analyse des données Vélib sur Python

In [ ]:
!pip install yellowbrick
!pip install prince

In [ ]:
import pandas as pd
import numpy as np
import random as rd
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.cluster import AgglomerativeClustering
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from scipy.spatial.distance import cdist
import scipy.cluster.hierarchy as sch
import yellowbrick
from yellowbrick.cluster import KElbowVisualizer
from yellowbrick.cluster import SilhouetteVisualizer
import csv
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.patches as mpatches
import matplotlib.colors as mcolors
from matplotlib import cm
import seaborn as sns
import plotly.express as px
from ipywidgets import interact, Dropdown, IntSlider
import prince
from prince import MCA

%matplotlib inline
plt.rcParams["font.size"] = 10
plt.rcParams["figure.figsize"] = (10, 5)

loading = pd.read_csv("data/velibLoading.csv", sep = " ")
coord = pd.read_csv("data/velibCoord.csv", sep = " ")

# I - Analyse exploratoire des données

## I.1 - Visualisation des données

In [ ]:
coord.head()

In [ ]:
loading.describe(include = "number")

In [ ]:
print(loading.shape)

## Imputation des données manquantes et unicité des individus

Nous allons nous assurer qu'il n'y ait pas de données manquantes ou de redondance dans nos individus qui introduiraient des biais dans notre analyse

In [ ]:
print("Il y a " + str(loading.isnull().any(axis = 0).sum()) + " ligne vide dans loading") # teste sur les lignes
print("Il y a " + str(loading.isnull().any(axis = 1).sum()) + " colonne vide dans loading") # teste sur les colonnes
print("")
print("Il y a " + str(coord.isnull().any(axis = 0).sum()) + " ligne vide dans coord") # teste sur les lignes
print("Il y a " + str(coord.isnull().any(axis = 1).sum()) + " colonne vide dans coord") # teste sur les colonnes
print("")
print("Il y a " + str(loading.duplicated().sum()) + " ligne identique dans loading") # teste sur loading
print("Il y a " + str(coord.duplicated().sum()) + " ligne identique dans coord") # teste sur coord

In [ ]:
# regroupe les lignes de coord par station
station_names = coord.names.value_counts().sort_values(ascending = False)
print(station_names)

## I.2 - Evolution du remplissage dans le temps

In [ ]:
stations = np.arange(loading.shape[0])
rd.shuffle(stations)
stations = stations[:4]

loading_data = loading.to_numpy()

n_steps = loading.shape[1]  # number of observed time steps
time_range = np.linspace(1, n_steps, n_steps)  # observed time range
time_tick  = np.linspace(1, n_steps, 8)  # beginning of days

fig, axs = plt.subplots(2, 2, figsize = (15,12))

for i in range(2):
    for j in range(2):
        k_station = stations[2 * i + j]
        axs[i, j].plot(time_range, loading_data[k_station, :], linewidth = 1, color = "blue")
        axs[i, j].vlines(x = time_tick, ymin = 0, ymax = 1, colors = "orange", linestyle = "dotted", linewidth = 3)
        axs[i, j].set_title(coord.names[1 + k_station], fontsize = 12)
        
for ax in axs.flat:
    ax.set_xlabel('Temps', fontsize = 12)
    ax.set_ylabel('Remplissage', fontsize = 12)
    ax.tick_params(axis='x', labelsize=10)
    ax.tick_params(axis='y', labelsize=10)
    ax.axvspan(0,6, color='lightblue', alpha=0.3)
    for i in range(n_steps):
        if i % 24 == 20:
            ax.axvspan(i,i+9, color='lightblue', alpha=0.5)
    ax.axvspan(120,168, color='lightpink', alpha=0.3) # tres visible sur canal saint denis
            
fig.suptitle("Disponibilité des Velibs en fonction du temps")    
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize = (20,6))

bp = plt.boxplot(loading_data, widths = 0.75, patch_artist = True)

for box in bp['boxes']:
    box.set_alpha(0.8)

for i,box in enumerate(bp['boxes']):
    box.set_alpha(0.5)
    if i//24 ==0:
        box.set_color('#FF0000')
    elif i//24 ==1:
        box.set_color('#FFA500')
    elif i//24 ==2:
        box.set_color('#FFFF00')
    elif i//24 ==3:
        box.set_color('#00FF00')
    elif i//24 ==4:
        box.set_color('#0000FF')
    elif i//24 ==5:
        box.set_color('#4B0082')
    else:
        box.set_color('#800080')
    
for median in bp['medians']:
    median.set(color = "black", linewidth=5)
    

plt.xlabel('Heures', fontsize = 20)
plt.ylabel('Remplissage', fontsize = 20)
plt.title("Boxplots", fontsize = 25)
plt.xticks(ticks = np.arange(0, 168, 5), labels=np.arange(0, 168, 5), fontsize = 15)
plt.yticks(fontsize = 15)

plt.tight_layout()
plt.show()

## I.3 - Comportement moyen du remplissage

In [ ]:
print('--- Average fill rate ---')
print(loading.mean().mean())

print('')

loading_mean = pd.Series(loading.mean(axis=1))

print('--- Least crowded station, on average ---')
i = loading_mean.idxmin()
print('Average fill rate :',loading_mean[i])
print(coord.loc[i])

print('')

print('--- Fullest station, on average ---')
i = pd.Series(loading.mean(axis=1)).idxmax()
print('Average fill rate :',loading.mean(axis=1)[i])
print(coord.loc[i])

In [ ]:
n_stations = loading.shape[0]  # number of observed stations
stations   = np.arange(n_stations)

plt.figure(figsize = (20,6))

plt.plot(loading_mean)
plt.hlines(y = loading.mean().mean(), xmin=0, xmax=n_stations, 
           colors = "OrangeRed", linewidth = 3)

plt.xlabel('Stations', fontsize = 20)
plt.ylabel('Remplissage moyen', fontsize = 20)
plt.title('Remplissage moyen par station', fontsize = 25)
plt.xticks(fontsize = 15)
plt.yticks(fontsize = 15)
plt.tight_layout()
plt.show()

In [ ]:
mean_per_hour_per_day = loading.mean(axis = 0).to_numpy()
mean_per_hour_per_day = mean_per_hour_per_day.reshape((7, 24))

mean_per_hour = mean_per_hour_per_day.mean(axis=0)

days = ["Lundi", "Mardi", "Mercredi","Jeudi", "Vendredi", "Samedi", "Dimanche"]
plt.figure(figsize = (15,10))

plt.plot(np.arange(1,25), mean_per_hour_per_day.transpose())
plt.plot(np.arange(1,25), mean_per_hour, color = "black", linewidth = 3)

plt.xlabel('Heures', fontsize = 20)
plt.ylabel('Remplissage', fontsize = 20)
plt.legend(days + ['Semaine'])
plt.xticks(ticks = np.arange(1,25), labels=np.arange(1,25), fontsize = 15)
  
plt.tight_layout
plt.show()

## I.4 - Répartition spatiale des stations

In [ ]:
hours = [6, 12, 23]
s, n = 10, len(hours)
fig, axs = plt.subplots(1, n, figsize = (s*n, s))

for (i,h) in enumerate(hours):
    im = axs[i].scatter(coord.latitude, coord.longitude, c = loading_data[:,h], cmap = cm.plasma_r)
    axs[i].set_title('Remplissage des stations - Lundi {} h'.format(h), fontsize = 25)
    plt.colorbar(im, ax=axs[i])
        
for ax in axs.flat:
    ax.set_xlabel('Latitude', fontsize = 20)
    ax.set_ylabel('Longitude', fontsize = 20)
    ax.tick_params(axis='x', labelsize=15)
    ax.tick_params(axis='y', labelsize=15)

plt.tight_layout()
plt.show()

In [ ]:
hours = [9, 10, 11]
s, n = 10, len(hours)
fig, axs = plt.subplots(1, n, figsize = (s*n, s))

for (i,h) in enumerate(hours):
    im = axs[i].scatter(coord.latitude, coord.longitude, c = loading_data[:,h], cmap = cm.plasma_r)
    axs[i].set_title('Remplissage des stations - Lundi {} h'.format(h), fontsize = 25)
    plt.colorbar(im, ax=axs[i])
        
for ax in axs.flat:
    ax.set_xlabel('Latitude', fontsize = 20)
    ax.set_ylabel('Longitude', fontsize = 20)
    ax.tick_params(axis='x', labelsize=15)
    ax.tick_params(axis='y', labelsize=15)

plt.tight_layout()
plt.show()

hours = [19, 20, 21]
s, n = 10, len(hours)
fig, axs = plt.subplots(1, n, figsize = (s*n, s))

for (i,h) in enumerate(hours):
    im = axs[i].scatter(coord.latitude, coord.longitude, c = loading_data[:,h], cmap = cm.plasma_r)
    axs[i].set_title('Remplissage des stations - Lundi {} h'.format(h), fontsize = 25)
    plt.colorbar(im, ax=axs[i])
        
for ax in axs.flat:
    ax.set_xlabel('Latitude', fontsize = 20)
    ax.set_ylabel('Longitude', fontsize = 20)
    ax.tick_params(axis='x', labelsize=15)
    ax.tick_params(axis='y', labelsize=15)

plt.tight_layout()
plt.show()

In [ ]:
h = 10
hours = np.arange(h, 168, 24)

load_per_hour = loading_data[:, hours]

days = ["Lundi", "Mardi", "Mercredi","Jeudi", "Vendredi", "Samedi", "Dimanche"]

s, m = 10, 3
k = 1 + len(days)//m

fig = plt.figure(figsize=(s*m, s*m))

for (i,d) in enumerate(days):
    ax = fig.add_subplot(k, m, i+1)
    im = ax.scatter(coord.latitude, coord.longitude, c = load_per_hour[:,i], cmap = cm.plasma_r)
    plt.colorbar(im)
    
    ax.set_title('Remplissage des stations - ' + d + ' {} h'.format(h), fontsize = 25)
    ax.set_xlabel('Latitude', fontsize = 20)
    ax.set_ylabel('Longitude', fontsize = 20)
    ax.tick_params(axis='x', labelsize=15)
    ax.tick_params(axis='y', labelsize=15)

plt.tight_layout()
plt.show()

In [ ]:
# Ne pas mettre Sunday 24h

hours = np.arange(0, 24)
days = ["Lundi", "Mardi", "Mercredi","Jeudi", "Vendredi", "Samedi", "Dimanche"]
s, n = 10, len(hours)

def plot_hour_day(hour, day):
    fig, ax = plt.subplots(figsize=(s, s))
    im = ax.scatter(coord.longitude, coord.latitude, c=loading_data[:, hour + 24*(days.index(day))], cmap=cm.plasma_r)
    ax.set_title('Remplissage des stations - {} {} h'.format(day, hour), fontsize=25)
    ax.set_xlabel('Longitude', fontsize=20)
    ax.set_ylabel('Latitude', fontsize=20)
    ax.tick_params(axis='both', labelsize=15)
    plt.colorbar(im, ax=ax)
    plt.show()

interact(plot_hour_day, hour=IntSlider(min=1, max=24, step=1, value=1), day=Dropdown(options=days, value='Lundi', description='Day:'))

In [ ]:
h = 8
hours = np.arange(h, 168, 24)
load_per_hour = loading_data[:, hours].mean(axis=1)

fig = px.scatter_mapbox(coord, lat = 'latitude', lon = 'longitude', 
                        mapbox_style = "carto-positron",
                        color = load_per_hour, color_continuous_scale = px.colors.sequential.Plasma_r, #size = load_per_hour,
                        zoom  = 10, opacity = .9,
                        title = 'Stations loading - Weekly average at {} h'.format(h))

fig.show()

In [ ]:
h = 18
hours = np.arange(h, 168, 24)
load_per_hour = loading_data[:, hours].mean(axis=1)

fig = px.scatter_mapbox(coord, lat = 'latitude', lon = 'longitude', 
                        mapbox_style = "carto-positron",
                        color = load_per_hour, color_continuous_scale = px.colors.sequential.Plasma_r, #size = load_per_hour,
                        zoom  = 10, opacity = .9,
                        title = 'Stations loading - Weekly average at {} h'.format(h))

fig.show()

In [ ]:
load_per_hour = loading_data[:, :].mean(axis=1)

fig = px.scatter_mapbox(coord, lat = 'latitude', lon = 'longitude', 
                        mapbox_style = "carto-positron",
                        color = load_per_hour, color_continuous_scale = px.colors.sequential.Plasma_r, #size = load_per_hour,
                        zoom  = 10, opacity = .9,
                        title = 'Average stations loading')

fig.show()

## I.5 - Analyse d'individus spécifiques : le cas particulier de stations situés sur des collines

In [ ]:
nb_pente = sum(coord['bonus'] == 1)

no_hill = sum(coord['bonus'] == 0)

print(nb_pente)

print("Proportion des stations situées en altitude:")

print(nb_pente / (nb_pente + no_hill))

In [ ]:
coord['hill'] = coord['bonus'].astype('category') # convert to categorical

fig = px.scatter_mapbox(coord, lat = 'latitude', lon = 'longitude',
mapbox_style = "carto-positron",
color = 'hill',
color_discrete_map = {0:'midnightblue', 1:'plum'},
labels = {0: "not hill", 1: "hill"},
zoom = 10, opacity = .9,
title = 'Stations situées en altitude (hill)')

fig.show()

In [ ]:
coord_data = coord.to_numpy()
index_hill = []
for i in range(len(coord_data)):
    if coord_data[i][-3] == 1:
        index_hill.append(i)
index_hill = np.array(index_hill)

index_not_hill = []
for i in range(len(coord_data)):
    if coord_data[i][-3] == 0:
        index_not_hill.append(i)
index_not_hill = np.array(index_not_hill)

loading_hill = loading_data[index_hill]

loading_hill_mean = pd.Series(loading_hill.mean(axis=1))

n_stations = loading.shape[0] 
stations = np.arange(n_stations)

plt.figure(figsize = (20,6))


plt.plot(loading_mean)
plt.hlines(y = loading.mean().mean(), xmin=0, xmax=n_stations,
colors = "OrangeRed", linewidth = 3)

plt.plot(index_hill,loading_hill_mean, color = 'purple', linewidth = 3)
plt.hlines(y = loading_hill_mean.mean().mean(), xmin=0, xmax=n_stations,
colors = "r", linewidth = 3, linestyles = '--')

plt.xlabel('Stations', fontsize = 20)
plt.ylabel('Remplissage', fontsize = 20)
plt.title("Moyenne du remplissage par station sur les collines comparé au remplissage moyen", fontsize = 25)
plt.xticks(fontsize = 15)
plt.yticks(fontsize = 15)

plt.tight_layout()
plt.show()

print(loading_hill_mean.mean().mean())

# II - Analyse en Composantes Principales (ACP)

In [ ]:
ss = StandardScaler()
loading_scaled = ss.fit_transform(loading)
pca = PCA()
loading_pca = pca.fit_transform(loading_scaled)

In [ ]:
plt.figure(figsize = (10, 5))
explained_variance_ratio = 100*pca.explained_variance_ratio_

plt.subplot(1,2,1)
n_bars = 20
plt.bar(np.arange(1, n_bars+1), explained_variance_ratio[:n_bars], color='blue')
plt.xlabel("Nombre de composantes")
plt.ylabel("Percentage of explained variance")
plt.xlim(0, 10)

plt.subplot(1,2,2)
# Tracé du graphe de l'inertie des composantes principales
plt.plot(range(1,9), pca.explained_variance_ratio_[:8], marker='o', linestyle='--')
#plt.title('Variance expliquée par composante')
plt.xlabel('Composante principale')
plt.ylabel(u'Ratio de variance expliquée')
plt.xticks(range(1, 10))
plt.grid(True)
plt.show()
print("Pourcentage de l'inertie expliquée par les 3 premières composantes:", round(sum([explained_variance_ratio[i] for i in range(3)]),2))

In [ ]:
plt.boxplot(loading_pca[:,0:10])
plt.tight_layout()
plt.title("Boxplot des composantes principales")
plt.show()
print("Pourcentage de l'inertie expliquée par les 5 premières composantes:", round(sum([explained_variance_ratio[i] for i in range(5)]),2))

In [ ]:
fig = plt.figure(figsize=(20,7))
plt.title("Graphe des variables sur 3 plans factoriels")
coord1=pca.components_[0]*np.sqrt(pca.explained_variance_[0])
coord2=pca.components_[1]*np.sqrt(pca.explained_variance_[1])

ax = fig.add_subplot(1, 3, 1)
for i, j, nom in zip(coord1,coord2, loading.columns):
    plt.text(i, j, nom, fontsize = 5)
    plt.arrow(0,0,i,j,color='black')
plt.axis((-1.2,1.2,-1.2,1.2))

c=plt.Circle((0,0), radius=1, color='gray', fill=False)
plt.xlabel('Composante Principale 1')
plt.ylabel('Composante Principale 2')
ax.add_patch(c)

coord1=pca.components_[0]*np.sqrt(pca.explained_variance_[0])
coord3=pca.components_[2]*np.sqrt(pca.explained_variance_[2])

ax = fig.add_subplot(1, 3, 2)
for i, j, nom in zip(coord1,coord3, loading.columns):
    plt.text(i, j, nom, fontsize = 5)
    plt.arrow(0,0,i,j,color='black')
plt.axis((-1.2,1.2,-1.2,1.2))

c=plt.Circle((0,0), radius=1, color='gray', fill=False)
plt.xlabel('Composante Principale 1')
plt.ylabel('Composante Principale 3')
ax.add_patch(c)

coord2=pca.components_[1]*np.sqrt(pca.explained_variance_[1])
coord4=pca.components_[3]*np.sqrt(pca.explained_variance_[3])

ax = fig.add_subplot(1, 3, 3)
for i, j, nom in zip(coord2,coord3, loading.columns):
    plt.text(i, j, nom, fontsize = 5)
    plt.arrow(0,0,i,j,color='black')
plt.axis((-1.2,1.2,-1.2,1.2))

c=plt.Circle((0,0), radius=1, color='gray', fill=False)
plt.xlabel('Composante Principale 2')
plt.ylabel('Composante Principale 4')
ax.add_patch(c)
plt.show()

In [ ]:
plt.figure(figsize=(5,4))
time_tick = [1 + 24 * i for i in range(7)]  # Vecteur délimitant les jours
i=0
plt.plot(pca.components_[i,:-1]*np.sqrt(pca.explained_variance_[i]))
plt.xlabel(u"variables")
plt.ylabel(u"Corrélation avec dim " + str(i + 1))
plt.ylim(-1, 1)
plt.axhline(0, color='green')
for tick in time_tick:
    plt.axvline(tick, color='orange')
plt.title("Corrélation des variables avec la première dimension")
plt.show()


In [ ]:
def group_variable_labels(variable_names, group_size):
    labels = []
    for i in range(0, len(variable_names), group_size):
        group_labels = ', '.join(variable_names[i:i+group_size])
        labels.append(group_labels)
    return labels
def group_variable_colors(variable_names):
    colors = sns.color_palette("husl", n_colors=24)
    groups = [i % 24 for i in range(len(variable_names))]
    group_colors = {group: color for group, color in zip(set(groups), colors)}
    return [group_colors[group] for group in groups], group_colors

fig = plt.figure(figsize=(15, 4))
ax = fig.add_subplot(1, 3, 3)

colors, group_colors = group_variable_colors(loading.columns)
for i, (x, y) in enumerate(zip(coord1, coord2)):
    ax.scatter(x, y, color=colors[i], edgecolor='black', linewidth=1, s=100)

legend_handles = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color, markeredgecolor='black', markersize=10, label=f'Heure {group}') for group, color in group_colors.items()]
plt.legend(handles=legend_handles, title='Groupes', loc='center left',ncol=2, bbox_to_anchor=(1, 0.5))

c = plt.Circle((0, 0), radius=1, color='gray', fill=False)
ax.add_patch(c)

plt.xlim(-1.2, 1.2)
plt.ylim(-1.2, 1.2)

plt.axhline(0, color='grey', linewidth=0.5)
plt.axvline(0, color='grey', linewidth=0.5)

plt.xlabel('Dimension 1')
plt.ylabel(' Dimension 2')

ax2 = fig.add_subplot(1, 3, 1)
time_tick = [1 + 24 * i for i in range(7)]
i=1
plt.plot(pca.components_[i,:-1]*np.sqrt(pca.explained_variance_[i]))
plt.xlabel(u"variables")
plt.ylabel(u"Corrélation avec dim " + str(i + 1))
plt.ylim(-1, 1)
plt.axhline(0, color='green')
for tick in time_tick:
    plt.axvline(tick, color='orange')

plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
plt.subplot(1,2,2)
vacances_indices = set(range(120, 169)).union(set(filter(lambda x: x % 24 in range(19, 24), range(len(loading.columns)))))

vacances_color = 'red'
rest_color = 'green'

for i, (x, y) in enumerate(zip(coord1, coord3)):
    color = vacances_color if i in vacances_indices else rest_color
    plt.scatter(x, y, color=color, edgecolor='black', linewidth=1, s=100)

circle = plt.Circle((0, 0), radius=1, color='gray', fill=False)
plt.gca().add_artist(circle)

plt.xlim(-1.2, 1.2)
plt.ylim(-1.2, 1.2)

plt.axhline(0, color='grey', linewidth=0.5)
plt.axvline(0, color='grey', linewidth=0.5)

plt.xlabel('Composante 1')
plt.ylabel('Composante 3')
plt.grid(True)

plt.legend(handles=[plt.scatter([], [], color='red', label='Pas d\'activité professionnelle'),
plt.scatter([], [], color='green', label='Activité professionnelle')], title='')
plt.subplot(1,2,1)
time_tick = [1 + 24 * i for i in range(7)]
i=2
plt.plot(pca.components_[i,:-1]*np.sqrt(pca.explained_variance_[i]))
plt.xlabel(u"variables")
plt.ylabel(u"Corrélation avec dim " + str(i + 1))
plt.ylim(-1, 1)
plt.axhline(0, color='green')
for tick in time_tick:
    plt.axvline(tick, color='orange')

plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
loading_with_hill = pd.concat([loading, coord.iloc[:, 2]], axis=1)
ss = StandardScaler()
loading_with_hill_scaled = ss.fit_transform(loading_with_hill)
pca = PCA()
loading_with_hill_pca = pca.fit_transform(loading_with_hill_scaled)

loading_with_hill_pca_df = pd.DataFrame(loading_with_hill_pca[:, :2], columns=['Dim1', 'Dim2'])

palette = plt.get_cmap("Dark2")
fig = plt.figure()
c = loading_with_hill["bonus"]
sc = plt.scatter(x='Dim1', y='Dim2', c=c, data=loading_with_hill_pca_df, s=10, cmap = "viridis")
plt.xlabel('Composante 1')
plt.ylabel('Composante 2')
plt.title('Graphe des individus selon la valeur de la variable "bonus"')
legend = plt.legend(*sc.legend_elements(), title = "Colline", loc = "upper right")
fig.add_artist(legend)

plt.show()

# III - Clustering

## III.1 - K-Means appliquée aux données sans ACP préalable

In [ ]:
plt.figure(figsize = (5, 5))

k_max = 15

silhouette = []
for k in range(2, k_max):
    kmeans = KMeans(n_clusters=k, init='k-means++', n_init='auto')
    clusters_kmeans = kmeans.fit_predict(loading_scaled)
    silhouette.append( silhouette_score(loading_scaled, clusters_kmeans, metric='euclidean') )
silhouette = np.array(silhouette)
plt.scatter(1,0, color="blue")
plt.scatter(range(2, k_max), silhouette, color="blue")
plt.show()
plt.figure(figsize = (15, 5))

plt.subplot(1,2,1)
model = KMeans(init='k-means++', n_init='auto')
visualizer = KElbowVisualizer(model,k=(1,15))
visualizer.fit(loading_scaled)        


plt.subplot(1,2,2)
model = KMeans(init='k-means++', n_init='auto')
visualizer = KElbowVisualizer(model,k=(3,15))

visualizer.fit(loading_scaled)
visualizer.show() 

### Pour K = 2

In [ ]:
k=2
reskmeans = KMeans(n_clusters=k,init='k-means++', n_init='auto', random_state=0)
C=reskmeans.fit_predict(loading)

cmap = plt.get_cmap('Dark2',k)
plt.scatter(loading_pca[:, 0], loading_pca[:, 1], s=2, c = C,linewidths=3, alpha=1, cmap=cmap)
plt.title("Graphe des Individus - PCA")
plt.xlabel('Composante Principale 1')
plt.ylabel('Composante Principale 2')

In [ ]:
ConfusionMatrixDisplay(confusion_matrix(C, coord['bonus'])).plot()
plt.ylabel('Classe prédite')
plt.xlabel('Variable Bonus')
plt.title('Matrice de confusion entre KMeans et Bonus')
plt.show()

In [ ]:
barycentres = reskmeans.cluster_centers_
ig, axs = plt.subplots(1, 2, figsize = (10,5))

for i in range(2):
        station = barycentres[i]
        axs[i].plot(time_range, station, linewidth = 1, color = 'purple')
        axs[i].vlines(x = time_tick, ymin = 0, ymax = 1, colors = "orange", linestyle = "dotted", linewidth = 3)
        axs[i].set_title('barycentre du cluster'+ str(i+1), fontsize = 12)

for ax in axs.flat:
    ax.set_xlabel('Heures', fontsize = 12)
    ax.set_ylabel('Remplissage', fontsize = 12)
    ax.tick_params(axis='x', labelsize=10)
    ax.tick_params(axis='y', labelsize=10)
    
plt.tight_layout()
plt.show()

In [ ]:
for i in range(2):
    Cluster = np.where(reskmeans.labels_ == i)[0]
    timeTick = 1 + 24 * np.arange(0, 7)
    plt.figure()
    plt.boxplot(loading.iloc[Cluster, :], patch_artist=True, boxprops=dict(facecolor='blue'),flierprops = dict(marker='.', markersize=3))
    plt.title(str(i))
    plt.xticks(ticks=np.arange(1, 168, 9), labels=np.arange(1, 168, 9))
    for tick in timeTick:
        plt.axvline(x=tick, color='orange', linestyle='dashed', linewidth=2)
    plt.show()

### Pour K = 4

In [ ]:
k=4
reskmeans2 = KMeans(n_clusters=k,init='k-means++', n_init='auto', random_state=0)
C=reskmeans2.fit_predict(loading)
cmap = plt.get_cmap('Dark2',k)
plt.scatter(loading_pca[:, 0], loading_pca[:, 1], s=2, c = C,linewidths=3, alpha=1, cmap=cmap)
plt.title("Graphe des Individus - PCA")
plt.xlabel('Composante Principale 1')
plt.ylabel('Composante Principale 2')

In [ ]:
barycentres = reskmeans2.cluster_centers_
fig, axs = plt.subplots(2, 2, figsize = (10,5))
count=0
for i in range(2):
    for j in range(2):
        station = barycentres[count]
        axs[i,j].plot(time_range, station, linewidth = 1, color = 'purple')
        axs[i,j].vlines(x = time_tick, ymin = 0, ymax = 1, colors = "orange", linestyle = "dotted", linewidth = 3)
        axs[i,j].set_title('barycentre du cluster'+ str(count+1), fontsize = 12)
        count=count+1
for ax in axs.flat:
    ax.set_xlabel('Heures', fontsize = 12)
    ax.set_ylabel('Remplissage', fontsize = 12)
    ax.tick_params(axis='x', labelsize=10)
    ax.tick_params(axis='y', labelsize=10)
    
plt.tight_layout()
plt.show()

In [ ]:
for i in range(4):
    Cluster = np.where(reskmeans2.labels_ == i)[0]
    timeTick = 1 + 24 * np.arange(0, 7)
    plt.figure()
    plt.boxplot(loading.iloc[Cluster, :], patch_artist=True, boxprops=dict(facecolor='blue'),flierprops = dict(marker='.', markersize=3))
    plt.title(str(i))
    plt.xticks(ticks=np.arange(1, 168, 9), labels=np.arange(1, 168, 9))
    for tick in timeTick:
        plt.axvline(x=tick, color='orange', linestyle='dashed', linewidth=2)
    plt.show()

In [ ]:
fig = px.scatter_mapbox(coord, lat = 'latitude', lon = 'longitude', 
                        mapbox_style = "carto-positron",
                        color = C,
                        zoom  = 10, opacity = .9,
                        title = 'Visualisation des clusters sur la carte')
fig.show()

### Pour K = 6

In [ ]:
k=6
reskmeans3 = KMeans(n_clusters=k,init='k-means++', n_init='auto', random_state=0)
C=reskmeans3.fit_predict(loading)
kmeans6 = C
barycentres = reskmeans3.cluster_centers_

In [ ]:
plt.subplot(1,2,1)
cmap = plt.get_cmap('Dark2',k)
plt.scatter(loading_pca[:, 0], loading_pca[:, 1], s=2, c = C,linewidths=3, alpha=1, cmap=cmap)
plt.title("Graphe des Individus - PCA")
plt.xlabel('Composante Principale 1')
plt.ylabel('Composante principale 2')

plt.subplot(1,2,2)
plt.scatter(loading_pca[:, 0], loading_pca[:, 3], s=2, c = C,linewidths=3, alpha=1, cmap=cmap)
plt.title("Graphe des Individus - PCA")
plt.xlabel('Composante principale 1')
plt.ylabel('Composante principale 4')
plt.show()

In [ ]:
fig, axs = plt.subplots(3, 2, figsize = (10,10))
count=0
for i in range(3):
    for j in range(2):
        station = barycentres[count]
        axs[i,j].plot(time_range, station, linewidth = 1, color = 'purple')
        axs[i,j].vlines(x = time_tick, ymin = 0, ymax = 1, colors = "orange", linestyle = "dotted", linewidth = 3)
        axs[i,j].set_title('barycentre du cluster'+ str(count+1), fontsize = 12)
        count=count+1
for ax in axs.flat:
    ax.set_xlabel('Heures', fontsize = 12)
    ax.set_ylabel('Remplissage', fontsize = 12)
    ax.tick_params(axis='x', labelsize=10)
    ax.tick_params(axis='y', labelsize=10)
    
plt.tight_layout()
plt.show()

In [ ]:
for i in range(6):
    Cluster = np.where(reskmeans3.labels_ == i)[0]
    timeTick = 1 + 24 * np.arange(0, 7)
    plt.figure()
    plt.boxplot(loading.iloc[Cluster, :], patch_artist=True, boxprops=dict(facecolor='blue'),flierprops = dict(marker='.', markersize=3))
    plt.title(str(i))
    plt.xticks(ticks=np.arange(1, 168, 9), labels=np.arange(1, 168, 9))
    for tick in timeTick:
        plt.axvline(x=tick, color='orange', linestyle='dashed', linewidth=2)
    plt.show()

## III.2 - Utilsation des données avec transformation ACP

In [ ]:
loading_reduced = loading_pca[:,:5]

In [ ]:
plt.figure(figsize = (5, 5))

k_max = 15

silhouette = []
for k in range(2, k_max):
    kmeans = KMeans(n_clusters=k, init='k-means++', n_init='auto')
    clusters_kmeans = kmeans.fit_predict(loading_reduced)
    silhouette.append( silhouette_score(loading_reduced, clusters_kmeans, metric='euclidean') )
silhouette = np.array(silhouette)
plt.scatter(1,0, color="blue")
plt.scatter(range(2, k_max), silhouette, color="blue")
plt.show()
plt.figure(figsize = (15, 5))

plt.subplot(1,2,1)
model = KMeans(init='k-means++', n_init='auto')
visualizer = KElbowVisualizer(model,k=(1,15))
visualizer.fit(loading_reduced)        

plt.subplot(1,2,2)
model = KMeans(init='k-means++', n_init='auto')
visualizer = KElbowVisualizer(model,k=(3,15))

visualizer.fit(loading_reduced)
visualizer.show() 

In [ ]:
k=4

reskmeans_pca = KMeans(n_clusters=k,init='k-means++', n_init='auto', random_state=0)
C=reskmeans_pca.fit_predict(loading_reduced)
plt.subplot(1,2,1)
cmap = plt.get_cmap('Dark2',k)
plt.scatter(loading_pca[:, 0], loading_pca[:, 1], s=2, c = C,linewidths=3, alpha=1, cmap=cmap)
plt.title("Graphe des Individus - PCA")
plt.xlabel('Composante Principale 1')
plt.ylabel('Composante Principale 2')

plt.subplot(1,2,2)
plt.scatter(loading_pca[:, 1], loading_pca[:, 4], s=2, c = C,linewidths=3, alpha=1, cmap=cmap)
plt.title("Graphe des Individus - PCA")
plt.xlabel('Composante Principale 1')
plt.ylabel('Composante Principale 3')
plt.show()

In [ ]:
def set_cluster_loading(clu, clusters):

    indices_clu = np.where(clusters == clu)[0]

    rd.shuffle(indices_clu)
    stations = indices_clu[:9]

    loading_data = loading.to_numpy()

    n_steps = loading.shape[1]  # number of observed time steps
    time_range = np.linspace(1, n_steps, n_steps)  # observed time range
    time_tick  = np.linspace(1, n_steps, 8)  # beginning of days


    fig, axs = plt.subplots(3, 3, figsize = (15,12))



    for i in range(3):
        for j in range(3):
            k_station = stations[3 * i + j]
            axs[i, j].plot(time_range, loading_data[k_station, :], linewidth = 1, color = "blue")
            axs[i, j].vlines(x = time_tick, ymin = 0, ymax = 1, colors = "orange", linestyle = "dotted", linewidth = 3)
            axs[i, j].set_title(coord.names[1 + k_station], fontsize = 12)


    for ax in axs.flat:
        ax.set_xlabel('Temps', fontsize = 12)
        ax.set_ylabel('Remplissage', fontsize = 12)
        ax.tick_params(axis='x', labelsize=10)
        ax.tick_params(axis='y', labelsize=10)
        ax.axvspan(0,6, color='lightblue', alpha=0.3)
        for i in range(n_steps):
            if i % 24 == 20:
                ax.axvspan(i,i+9, color='lightblue', alpha=0.5)
        ax.axvspan(120,168, color='lightpink', alpha=0.3)

    fig.suptitle("Disponibilité des Velibs en fonction du temps")    
    plt.tight_layout()
    plt.show()


In [ ]:
set_cluster_loading(0, C)

In [ ]:
set_cluster_loading(1, C)

In [ ]:
set_cluster_loading(2, C)

In [ ]:
set_cluster_loading(3, C)

## III.3 - Classification ascendante hiérachique (CAH)

In [ ]:
ac = AgglomerativeClustering(linkage='ward', compute_distances=True)
plt.figure(figsize = (15, 5))
plt.subplot(1,2,1)
visualizer = KElbowVisualizer(ac, k=(1,15))
visualizer.fit(loading_scaled)  

plt.subplot(1,2,2)
visualizer = KElbowVisualizer(ac, k=(3,15))
visualizer.fit(loading_scaled)  
plt.show()

In [ ]:
ac = AgglomerativeClustering( compute_distances=True, linkage='ward')
clusters = ac.fit(loading)

children = ac.children_
distances = ac.distances_
n_observations = np.arange(2, children.shape[0]+2)
linkage_matrix = np.c_[children, distances, n_observations]

sch.dendrogram(linkage_matrix, labels=ac.labels_)

plt.title("Dendogramme avec liaison Ward")
plt.show()

In [ ]:
K = 6

sch.dendrogram(linkage_matrix, labels=ac.labels_)

max_d = .5*(ac.distances_[-K]+ac.distances_[-K+1])
plt.axhline(y=max_d, c='k')
plt.title("Dendogramme avec liaison Ward")
plt.show()

In [ ]:
plt.subplot(1,2,1)
cmap = plt.get_cmap('Dark2',K)
ac = AgglomerativeClustering(n_clusters=K, compute_distances=True, linkage='ward')
clusters_ac = ac.fit_predict(loading)
plt.scatter(loading_pca[:, 0], loading_pca[:, 1], s=2, c = clusters_ac,linewidths=3, alpha=1, cmap=cmap)
plt.title("Graphe des Individus - PCA")
plt.xlabel('Composante Principale 1')
plt.ylabel('Composante Principale 2')

In [ ]:
set_cluster_loading(0, clusters_ac)

In [ ]:
set_cluster_loading(1, clusters_ac)

In [ ]:
set_cluster_loading(2, clusters_ac)

In [ ]:
set_cluster_loading(3, clusters_ac)

In [ ]:
set_cluster_loading(4, clusters_ac)

In [ ]:
set_cluster_loading(5, clusters_ac)

In [ ]:
fig = px.scatter_mapbox(coord, lat = 'latitude', lon = 'longitude',
                        mapbox_style = "carto-positron",
                        color = clusters_ac,
                        zoom  = 10, opacity = .9,
                        title = 'Visualisation des clusters sur la carte')
fig.show()

## III.4 - Gaussin mixture models (GMM)

In [ ]:
k_max = 15
plt.figure(figsize = (15, 5))

bic = []
for k in range(2, k_max):
    gmm = GaussianMixture(n_components=k, init_params='kmeans', n_init=3)
    gmm.fit(loading_reduced)
    bic.append(gmm.bic(loading_reduced))
bic = np.array(bic)

plt.scatter(range(2, k_max), bic)
plt.title('Avec le critère BIC')

plt.show()

In [ ]:
k_max = 15

silhouette = []
for k in range(2, k_max):
    gmm = GaussianMixture(n_components=k, init_params='kmeans', n_init=1)
    clusters_gmm = gmm.fit_predict(loading_reduced)
    silhouette.append(silhouette_score(loading_reduced, clusters_gmm, metric='euclidean') )
silhouette = np.array(silhouette)

plt.scatter(range(2, k_max), silhouette)
plt.show()

In [ ]:
K=4
gmm = GaussianMixture(n_components=K, n_init=3)
clusters_gmm = gmm.fit_predict(loading_reduced)

In [ ]:
plt.subplot(1,2,1)
cmap = plt.get_cmap('Dark2',K)
plt.scatter(loading_pca[:, 0], loading_pca[:, 1], s=2, c = clusters_gmm,linewidths=3, alpha=1, cmap=cmap)
plt.title("Graphe des Individus - PCA")
plt.xlabel('Composante Principale 1')
plt.ylabel('Composante Principale 2')
plt.show()

In [ ]:
fig = px.scatter_mapbox(coord, lat = 'latitude', lon = 'longitude',
                        mapbox_style = "carto-positron",
                        color = clusters_gmm,
                        zoom  = 10, opacity = .9,
                        title = 'Visualisation des clusters sur la carte')
fig.show()

In [ ]:
set_cluster_loading(0, clusters_gmm)

In [ ]:
set_cluster_loading(1, clusters_gmm)

In [ ]:
set_cluster_loading(2, clusters_gmm)

In [ ]:
set_cluster_loading(3, clusters_gmm)

# IV - Comparaison des méthodes de classification

## IV.1 - Utilisation des données avec ou sans traitement d'ACP

In [ ]:
k = 4

loading = pd.read_csv("data/velibLoading.csv", sep = " ")
ss = StandardScaler()
loading_scaled = ss.fit_transform(loading)

pca = PCA()
loading_pca = pca.fit_transform(loading_scaled)
loading_reduced = loading_pca[:, :5]

reskmeans_pca = KMeans(n_clusters=k, init='k-means++', n_init='auto', random_state=42)
c_pca = reskmeans_pca.fit_predict(loading_reduced)

reskmeans_no_pca = KMeans(n_clusters=k, init='k-means++', n_init='auto', random_state=42)
c_no_pca = reskmeans_no_pca.fit_predict(loading)

ConfusionMatrixDisplay(confusion_matrix(c_pca, c_no_pca),display_labels=np.arange(1, k+1)).plot()
plt.ylabel('Classe prédite (avec ACP)')
plt.xlabel('Classe prédite (sans ACP)')
plt.title('Matrice de confusion entre KMeans avec et sans ACP')
plt.show()

In [ ]:
def matchClasses(classif1, classif2):
    cm = confusion_matrix(classif1, classif2)
    K = cm.shape[0]
    a, b = np.zeros(K), np.zeros(K)
    l = np.linspace(0,K-1,K)
    ind = 0
    for j in range(K):
        for i in range(K):
            if (a[j] < cm[i,j] and (i in l)):
                a[j] = cm[i,j]
                b[j] = i
                ind = i
        l = np.delete(l, np.where(l == ind)[0])
    a = a.astype(int)
    b = b.astype(int)
                                             
    print ("")
    print ("Classes size:", a)
    print ("Class (in the classif1 numbering):", b)
    print ("")
    
    table = cm.copy()
    for i in range(K):
        table[:,b[i]] = cm[:,i]   
        
    clusters = classif2.copy()
    n = classif2.shape[0]
    for i in range(n):
        for j in range(K):
            if (classif2[i] == j):
                clusters[i] = b[j]
                
    return table, clusters

In [ ]:
cm, clusters_kmeans_sorted = matchClasses(c_pca, c_no_pca)
ConfusionMatrixDisplay(cm,display_labels=np.arange(1, k+1)).plot()
plt.ylabel('Classe prédite (avec ACP)')
plt.xlabel('Classe prédite (sans ACP)')
plt.title('Matrice de confusion réordonnée entre KMeans avec et sans ACP')
plt.show()

In [ ]:
cm, clusters_sorted_kmeans4nopca = matchClasses(c_pca, c_no_pca)

cm, clusters_gmm_sorted = matchClasses(c_pca, clusters_gmm)

cm2, clusters_ac_sorted= matchClasses(kmeans6, clusters_ac)

## IV.2 - Analyse en comparaisons multiples (MCA)

In [ ]:
data_clusters = {
    'km2c': reskmeans.labels_,
    'km4c': clusters_sorted_kmeans4nopca,
    'km6c': reskmeans3.labels_,  
    'kmPCA4c': reskmeans_pca.labels_,
    'cahC': clusters_ac_sorted,  
    'gmmC': clusters_gmm_sorted, 
    'hill': coord['bonus']
}

df_clusters = pd.DataFrame(data_clusters)
df_clusters.iloc[:, :7] = df_clusters.iloc[:, :7].astype('category')

print(df_clusters.dtypes)
df_clusters.info()


In [ ]:
df_clusters_quali=df_clusters.iloc[:, :6]

res_mca = MCA(n_components=9,n_iter=10,copy=True,check_input=True,engine='sklearn',random_state=42)
res_mca.fit(df_clusters_quali)  # Sélection des 6 premières colonnes pour l'analyse MCA

display(res_mca.eigenvalues_summary)

res_mca.scree_plot()

In [ ]:
res_mca.plot(
    df_clusters_quali,
    x_component=0,
    y_component=1,
    show_column_markers=True,
    show_row_markers=False,
    show_column_labels=True,)

In [ ]:
res_mca.column_cosine_similarities(df_clusters_quali).head()

In [ ]:
cos2_dim1 = res_mca.column_cosine_similarities(df_clusters.iloc[:, :6]).iloc[:, 0]

cos2_dim1_sorted = cos2_dim1.sort_values(ascending=False)

plt.figure(figsize=(10, 6))
plt.bar(cos2_dim1_sorted.index, cos2_dim1_sorted.values, color='salmon')
plt.xlabel('Variables')
plt.ylabel('Cos2 Values (Dimension 1)')
plt.title('Cos2 Values for Variables (Dimension 1 - Sorted)')
plt.xticks(rotation=45, ha='right')
plt.grid(True)
plt.show()

In [ ]:
res_mca.plot(
    df_clusters_quali,
    x_component=0,
    y_component=1,
    show_column_markers=False,
    show_row_markers=True,
    show_column_labels=False,
    show_row_labels=True
)

In [ ]:
contrib = res_mca.column_contributions_.style.format('{:.1%}')
display(contrib.highlight_max(color='orange').highlight_min(color='lightblue'))


############### Contributions liées a la premiere dimension

contrib1 = res_mca.column_contributions_.iloc[:, 0]

contrib1_sorted = contrib1.sort_values(ascending=False)

plt.figure(figsize=(10, 6))
plt.bar(contrib1_sorted.index, contrib1_sorted.values, color='skyblue')
plt.title('Pourcentages des contributions des variables selon la composante 1')
plt.xlabel('Variables')
plt.ylabel('% de contribution')
plt.xticks(rotation=45, ha='right')  # Rotation des étiquettes de l'axe des x
plt.grid(axis='y', linestyle='--', alpha=0.7)  # Affichage d'une grille en pointillés sur l'axe y
plt.show()

############### Contributions liées a la seconde dimension

contrib2 = res_mca.column_contributions_.iloc[:, 1]

contrib2_sorted = contrib2.sort_values(ascending=False)

plt.figure(figsize=(10, 6))
plt.bar(contrib2_sorted.index, contrib2_sorted.values, color='skyblue')
plt.title('Pourcentages des contributions des variables selon la composante 2')
plt.xlabel('Variables')
plt.ylabel('% de contribution')
plt.xticks(rotation=45, ha='right')  # Rotation des étiquettes de l'axe des x
plt.grid(axis='y', linestyle='--', alpha=0.7)  # Affichage d'une grille en pointillés sur l'axe y
plt.show()

In [ ]:
res_mca.plot(
    df_clusters.iloc[:, :7],
    x_component=0,
    y_component=1,
    show_column_markers=True,
    show_row_markers=False,
    show_column_labels=True,
    show_row_labels=False
)

In [ ]:
column_coords = res_mca.column_coordinates(df_clusters.iloc[:, :7])

colors = list(mcolors.TABLEAU_COLORS.values()) + list(mcolors.XKCD_COLORS.values())[:20] 
palette = mcolors.ListedColormap(colors)

plt.figure(figsize=(10, 8))

for i, variable in enumerate(df_clusters.iloc[:, :7].columns):
    modalities_coords = column_coords[column_coords.index.str.startswith(variable)]
    plt.scatter(modalities_coords.iloc[:, 0], modalities_coords.iloc[:, 1], label=variable, color=palette(i))

plt.axhline(0, color='black', linestyle='--', linewidth=0.5)
plt.axvline(0, color='black', linestyle='--', linewidth=0.5)

plt.title('Corrélation entre les variables et les axes principaux')
plt.xlabel('Composante principale 1')
plt.ylabel('Composante principale 2')

plt.legend()
plt.grid(True)
plt.show()

# V - MCA sur le jeu de données complet

In [ ]:
x = loading.copy()
n1 = {}
n2 = {}
Xcat = coord['bonus'].astype('category')
day = np.array(list(range(7, 18+1)))
night = np.array(list(range(0, 7)) + list(range(19, 22+1)))
labels = ["a", "b", "c"]
row_means = np.mean(x, axis=1)
quantiles = np.quantile(row_means, [1/3, 2/3])
breaks = np.concatenate(([-0.01], quantiles, [1.01]))
for i in range(1, 8):
    ind = list(day + (i-1)*24)
    newCol = np.mean(x.iloc[:, ind], axis=1)
    quantiles = np.quantile(newCol, [1/3, 2/3])
    newCol = pd.cut(newCol, bins=breaks, labels=labels)
    newName = 'day' + str(i)
    n1[newName] = newCol
Xcat = pd.concat([Xcat, pd.DataFrame(n1)], axis=1)
 
for i in range(1, 8):
    ind = list(night + (i-1)*24)
    newCol = np.mean(x.iloc[:,ind], axis=1)
    quantiles = np.quantile(newCol, [1/3, 2/3])
    newCol = pd.cut(newCol, bins=breaks, labels=labels)
    newName = 'night' + str(i)
    n2[newName] = newCol
Xcat = pd.concat([Xcat, pd.DataFrame(n2)], axis=1)

In [ ]:
Xcat.head()

In [ ]:
res2_mca = MCA(n_components=9,n_iter=10,copy=True,check_input=True,engine='sklearn',random_state=42)
res2_mca.fit(Xcat)

display(res2_mca.eigenvalues_summary)

res_mca.scree_plot()

In [ ]:
res2_mca.plot(
    Xcat,
    x_component=0,
    y_component=1,
    show_column_markers=True,
    show_row_markers=False,
    show_column_labels=True,
    show_row_labels=False
)

In [ ]:
res2_mca.plot(
    Xcat,
    x_component=0,
    y_component=2,
    show_column_markers=True,
    show_row_markers=False,
    show_column_labels=True,
    show_row_labels=False
)

In [ ]:
column_coords = res2_mca.column_coordinates(Xcat)

colors = list(mcolors.TABLEAU_COLORS.values()) + list(mcolors.XKCD_COLORS.values())[:20]
palette = mcolors.ListedColormap(colors)

plt.figure(figsize=(10, 8))

for i, variable in enumerate(Xcat.columns):
    modalities_coords = column_coords[column_coords.index.str.startswith(variable)]
    plt.scatter(modalities_coords.iloc[:, 0], modalities_coords.iloc[:, 1], label=variable, color=palette(i))

plt.axhline(0, color='black', linestyle='--', linewidth=0.5)
plt.axvline(0, color='black', linestyle='--', linewidth=0.5)

plt.title('Corrélation entre les variables et les axes principaux')
plt.xlabel('Composante principale 1')
plt.ylabel('Composante principale 2')

plt.legend()
plt.grid(True)
plt.show()

In [ ]:
res2_mca.plot(
    Xcat,
    x_component=0,
    y_component=1,
    show_column_markers=False,
    show_row_markers=True,
    show_column_labels=False,
    show_row_labels=True
)

In [ ]:
cos2_dim1 = res2_mca.column_cosine_similarities(Xcat).iloc[:, 0]

cos2_dim1_sorted = cos2_dim1.sort_values(ascending=False)

plt.figure(figsize=(10, 6))
plt.bar(cos2_dim1_sorted.index, cos2_dim1_sorted.values, color='salmon')
plt.xlabel('Variables')
plt.ylabel('Cos2 Values (Dimension 1)')
plt.title('Cos2 Values for Variables (Dimension 1 - Sorted)')
plt.xticks(rotation=45, ha='right') 
plt.grid(True)
plt.show()


In [ ]:
contrib = res2_mca.column_contributions_.style.format('{:.1%}')
display(contrib.highlight_max(color='orange').highlight_min(color='lightblue'))